In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langsmith import Client, traceable
from typing_extensions import Annotated, TypedDict

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
# List of URLs to load documents from
#urls = [
#    "https://admisiones.unal.edu.co/pregrado/",
#    "https://admisiones.unal.edu.co/pregrado/admision-a-pregrado/",
#    "https://admisiones.unal.edu.co/pregrado/admision-a-los-programas-curriculares-con-prueba-especifica-musica-musica-instrumental-artes-plasticas-y-cine-y-television/",
#]

from data.urls import WEB_PATHS

# Load documents from the URLs
docs = [WebBaseLoader(url).load() for url in WEB_PATHS]
docs_list = [item for sublist in docs for item in sublist]

# Initialize a text splitter with specified chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

# Split the documents into chunks
doc_splits = text_splitter.split_documents(docs_list)

# Add the document chunks to the "vector store" using OpenAIEmbeddings
vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits,
    embedding=OpenAIEmbeddings(),
)

# With langchain we can easily turn any vector store into a retrieval component:
retriever = vectorstore.as_retriever(k=6)

In [ ]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3.2")

# Add the document chunks to the "vector store" using OpenAIEmbeddings
vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits,
    embedding=embeddings,
)

# With langchain we can easily turn any vector store into a retrieval component:
retriever = vectorstore.as_retriever(k=6)

In [15]:
from langchain_ollama import ChatOllama
from langsmith import traceable


llm_llama = ChatOllama(
    model="llama3.2",
    temperature=0,
)

# Add decorator so this function is traced in LangSmith
@traceable()
def rag_bot_llama(question: str) -> dict:
    # LangChain retriever will be automatically traced
    docs = retriever.invoke(question)
    docs_string = "\n\n".join(doc.page_content for doc in docs)

    instructions = f"""You are a helpful assistant who is good at analyzing source information and answering questions.       Use the following source documents to answer the user's questions.       If you don't know the answer, just say that you don't know.       Use three sentences maximum and keep the answer concise.

Documents:
{docs_string}"""

    # langchain ChatModel will be automatically traced
    ai_msg = llm_llama.invoke([
            {"role": "system", "content": instructions},
            {"role": "user", "content": question},
        ],
    )

    return {"answer": ai_msg.content, "documents": docs}

In [5]:
from langchain_openai import ChatOpenAI
from langsmith import traceable

llm_gpt = ChatOpenAI(model="gpt-4o-mini", temperature=1)


# Add decorator so this function is traced in LangSmith
@traceable()
def rag_bot_openai(question: str) -> dict:
    # LangChain retriever will be automatically traced
    docs = retriever.invoke(question)
    docs_string = "\n\n".join(doc.page_content for doc in docs)

    instructions = f"""You are a helpful assistant who is good at analyzing source information and answering questions.       Use the following source documents to answer the user's questions.       If you don't know the answer, just say that you don't know.       Use three sentences maximum and keep the answer concise.

Documents:
{docs_string}"""

    # langchain ChatModel will be automatically traced
    ai_msg = llm_gpt.invoke([
            {"role": "system", "content": instructions},
            {"role": "user", "content": question},
        ],
    )

    return {"answer": ai_msg.content, "documents": docs}

# Dataset

In [6]:
from langsmith import Client
from src.qa_dataset import examples

client = Client()



# Create the dataset and examples in LangSmith
dataset_name = "Admisiones Q&A pre y pos"
dataset = client.create_dataset(dataset_name=dataset_name)
client.create_examples(
    dataset_id=dataset.id,
    examples=examples
)

{'example_ids': ['913bd733-976c-4e6f-ad44-879fe29ba915',
  '7c123c68-da95-4382-83c1-c987ac6ede60',
  '29818cf6-0316-4b6b-875c-0bec7b2c51c9',
  '5fdd5eaa-0c55-4368-b88c-14f6737c0415',
  'ca8995af-d440-481b-b96f-9beed3fe9cb6',
  '4870a26d-9d5b-4371-a331-622f016f9f96',
  '5dd52191-2310-4b2f-a363-e07d07350642',
  '45125c16-301e-4b1d-a871-3b5a8dc43cc9',
  '48eea4b0-aa5e-4e8f-94c9-d3cfc42af137',
  '209fbff6-a341-4e94-b638-530dc926a061',
  '49e7ec2e-6460-4c0a-886c-9c1f0d1570c8',
  '90e1bab7-8bbf-4203-b020-c116c0f0a36c',
  '859d109e-9b9e-468a-bb73-25ce3e0d028d',
  '982b52ba-71b3-4af9-be9c-87820799d118',
  'a7ffe77d-7bd6-438d-8569-7d0dfddd0a88',
  'fba90595-a6e9-4e89-aed4-488449a9edda',
  '4c7e0b65-7cb2-4886-8292-c572eaecd3e9',
  'b16fb0d5-e998-4d39-8e01-8f015faa5b03',
  '34960488-b533-4944-91cd-32bae8f8c205',
  '859c1420-4925-4c0f-be48-5b0fe48882e1',
  '99909f4d-e1c6-4328-a52d-90d0da5dbde4',
  'f4bc14a9-0e39-4402-b0c7-d25a4d4b7fd2',
  '74212ebf-2418-4d64-9d3f-fda553a7c099',
  'c0e16ba4-4135-4d

# Correctness: Response vs reference answer

In [7]:
from typing_extensions import Annotated, TypedDict

# Grade output schema
class CorrectnessGrade(TypedDict):
    # Note that the order in the fields are defined is the order in which the model will generate them.
    # It is useful to put explanations before responses because it forces the model to think through
    # its final response before generating it:
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    correct: Annotated[bool, ..., "True if the answer is correct, False otherwise."]

# Grade prompt
correctness_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer. 
(2) Ensure that the student answer does not contain any conflicting statements.
(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the  ground truth answer.

Correctness:
A correctness value of True means that the student's answer meets all of the criteria.
A correctness value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM
grader_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(CorrectnessGrade, method="json_schema", strict=True)

def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """An evaluator for RAG answer accuracy"""
    answers = f"""\
QUESTION: {inputs['question']}
GROUND TRUTH ANSWER: {reference_outputs['answer']}
STUDENT ANSWER: {outputs['answer']}"""

    # Run evaluator
    grade = grader_llm.invoke([
        {"role": "system", "content": correctness_instructions}, 
        {"role": "user", "content": answers}
    ])
    return grade["correct"]

# Relevance: Response vs input

In [8]:
# Grade output schema
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "Provide the score on whether the answer addresses the question"]

# Grade prompt
relevance_instructions="""You are a teacher grading a quiz. 

You will be given a QUESTION and a STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is concise and relevant to the QUESTION
(2) Ensure the STUDENT ANSWER helps to answer the QUESTION

Relevance:
A relevance value of True means that the student's answer meets all of the criteria.
A relevance value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Relevance LLM
relevance_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(RelevanceGrade, method="json_schema", strict=True)

# Evaluator
def relevance(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer helpfulness."""
    answer = f"QUESTION: {inputs['question']}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = relevance_llm.invoke([
        {"role": "system", "content": relevance_instructions}, 
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

# Groundedness: Response vs retrieved docs

In [9]:
# Grade output schema
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[bool, ..., "Provide the score on if the answer hallucinates from the documents"]

# Grade prompt
grounded_instructions = """You are a teacher grading a quiz. 

You will be given FACTS and a STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is grounded in the FACTS. 
(2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Grounded:
A grounded value of True means that the student's answer meets all of the criteria.
A grounded value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM 
grounded_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(GroundedGrade, method="json_schema", strict=True)

# Evaluator
def groundedness(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer groundedness."""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = grounded_llm.invoke([{"role": "system", "content": grounded_instructions}, {"role": "user", "content": answer}])
    return grade["grounded"]

# Retrieval relevance: Retrieved docs vs input

In [10]:
# Grade output schema
class RetrievalRelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "True if the retrieved documents are relevant to the question, False otherwise"]

# Grade prompt
retrieval_relevance_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION and a set of FACTS provided by the student. 

Here is the grade criteria to follow:
(1) You goal is to identify FACTS that are completely unrelated to the QUESTION
(2) If the facts contain ANY keywords or semantic meaning related to the question, consider them relevant
(3) It is OK if the facts have SOME information that is unrelated to the question as long as (2) is met

Relevance:
A relevance value of True means that the FACTS contain ANY keywords or semantic meaning related to the QUESTION and are therefore relevant.
A relevance value of False means that the FACTS are completely unrelated to the QUESTION.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM
retrieval_relevance_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(RetrievalRelevanceGrade, method="json_schema", strict=True)

def retrieval_relevance(inputs: dict, outputs: dict) -> bool:
    """An evaluator for document relevance"""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nQUESTION: {inputs['question']}"

    # Run evaluator
    grade = retrieval_relevance_llm.invoke([
        {"role": "system", "content": retrieval_relevance_instructions}, 
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

In [11]:
def target(inputs: dict) -> dict:
    return rag_bot_openai(inputs["question"])

experiment_results = client.evaluate(
    target,
    data=dataset_name,
    evaluators=[correctness, groundedness, relevance, retrieval_relevance],
    experiment_prefix="rag-doc-relevance-gpt4omini",
    metadata={"version": "nachabot0.1-gpt4o-mini"},
)
# Explore results locally as a dataframe if you have pandas installed
# experiment_results.to_pandas()

/Users/ctafurdevia/.pyenv/versions/3.10.15/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'rag-doc-relevance-gpt4omini-9af7615f' at:
https://smith.langchain.com/o/fdbe245f-39cf-4560-bd35-48e47f7a1d78/datasets/c17dedc2-389b-4443-bc57-0d6daffa14d0/compare?selectedSessions=350f5449-c394-492a-92db-6fa36dd20f51




42it [10:15, 14.66s/it]


In [ ]:
def target_llama(inputs: dict) -> dict:
    return rag_bot_llama(inputs["question"])

experiment_results_llama = client.evaluate(
    target_llama,
    data=dataset_name,
    evaluators=[correctness, groundedness, relevance, retrieval_relevance],
    experiment_prefix="rag-doc-relevance-llama3.2",
    metadata={"version": "nachabot0.2-llama3.2"},
)

View the evaluation results for experiment: 'rag-doc-relevance-llama3.2-e853d6a0' at:
https://smith.langchain.com/o/fdbe245f-39cf-4560-bd35-48e47f7a1d78/datasets/c17dedc2-389b-4443-bc57-0d6daffa14d0/compare?selectedSessions=339eeb10-6c5e-47ba-9d1b-4ff82ae4f6b1




42it [09:46, 13.97s/it]


In [16]:
experiment_results_llama_emb = client.evaluate(
    target_llama,
    data=dataset_name,
    evaluators=[correctness, groundedness, relevance, retrieval_relevance],
    experiment_prefix="rag-doc-relevance-llama3.2-emb",
    metadata={"version": "nachabot0.3-llama3.2-emb"},
)

View the evaluation results for experiment: 'rag-doc-relevance-llama3.2-emb-202b0529' at:
https://smith.langchain.com/o/fdbe245f-39cf-4560-bd35-48e47f7a1d78/datasets/c17dedc2-389b-4443-bc57-0d6daffa14d0/compare?selectedSessions=56ee16fe-b672-4e6d-9c67-3eb97690b7bf




42it [09:00, 12.88s/it]


In [17]:
experiment_results_llama.to_pandas()

,inputs.question,outputs.answer,outputs.documents,error,reference.answer,feedback.correctness,feedback.groundedness,feedback.relevance,feedback.retrieval_relevance,execution_time,example_id,id
0,¿Puedo cambiar la hora de presentación de la p...,"No, no se puede cambiar la hora de presentació...",[page_content='Presentación de la prueba de ad...,None,NO. La prueba de admisión se realiza de forma ...,True,True,True,True,4.271412,00d4b17d-38bd-4944-b58d-e23647884514,b0936ad7-0be7-454e-9c2c-c0004e4931d7
1,En caso de ser admitido y no tener el acta de ...,No tengo información específica sobre el plazo...,[page_content='entregar los documentos para la...,None,Se invita a consultar con la División de Regis...,True,True,True,True,3.860412,0d4fdcce-cd78-4f59-9d51-746bc92e032b,303478db-6cc0-4b03-86c6-c141db2521d1
2,En mi profesión no existe Registro Médico ni ...,No tengo información específica sobre cómo dil...,[page_content='tipo de discapacidad en el form...,None,Los datos relacionados con la Tarjeta Profesio...,False,False,True,False,2.869682,1400d6a6-426e-49a8-acd4-f8bad69a0ea6,0a614a35-7c76-40b6-8346-173d054f2126
3,¿Qué pasa si un aspirante no inscribe programa...,Si un aspirante no inscribe programa curricula...,[page_content='34. Si el aspirante es convocad...,None,"Si el aspirante presentó la prueba, fue habili...",False,True,True,True,2.977211,209fbff6-a341-4e94-b638-530dc926a061,969ee2b7-3afd-4b14-a51d-cd9d905eb9a7
4,¿Qué es la admisión a pregrado?,La admisión a pregrado es el proceso mediante ...,[page_content='Admisión a pregrado\n\t\t\t\nLa...,None,La admisión es el proceso mediante el cual se ...,True,False,True,True,3.330625,29818cf6-0316-4b6b-875c-0bec7b2c51c9,89f43bfa-5196-4591-82b0-6f636a058ec4
5,Si el programa al que deseo presentarme exige ...,No tengo información específica sobre cómo pro...,[page_content='Los aspirantes a los programas ...,None,Una vez usted realiza inscripción al proceso d...,False,False,False,True,3.956207,2b38f89d-b44b-42f3-894b-dcf8c80c1eef,e38ad755-8069-4cc9-9680-40f6af804539
6,¿Cuál es la duración de la presentación de la ...,No tengo información específica sobre la durac...,[page_content='Presentación de la prueba de ad...,None,La prueba de admisión tiene una duración de 3 ...,False,True,True,False,2.281535,2f0beff2-61b6-4fd0-9c96-b652f63f7ce3,9aaf76c3-4f36-422b-b656-e9962bee7b66
7,En caso de ser admitido a un programa de posgr...,"Para ser admitido a un programa de posgrado, e...",[page_content='con la Coordinación Curricular ...,None,La información sobre trámites y calendario par...,True,True,True,True,3.632357,34960488-b533-4944-91cd-32bae8f8c205,28c9747b-b2d0-4200-85a5-14e6b1505ef7
8,No tengo número de registro del Examen de Esta...,"Como extranjero, no necesitas un número de reg...",[page_content='debe identificarse con Cédula d...,None,"En el formulario electrónico de inscripción, e...",True,False,True,True,3.023427,3b174356-4c21-44f6-9a59-c546bb777949,272a9a14-7efa-4c1e-a4fb-82b914a9101e
9,¿Debe el aspirante ser bachiller o haber prese...,"No, el aspirante no necesita ser bachiller ni ...","[page_content='undécimo, puedo inscribirme y p...",None,No. Para presentar la prueba de admisión el as...,True,True,True,True,3.297663,45125c16-301e-4b1d-a871-3b5a8dc43cc9,e8b261eb-fee7-4856-a706-efe3fe0309b5


In [13]:
experiment_results.to_pandas()

,inputs.question,outputs.answer,outputs.documents,error,reference.answer,feedback.correctness,feedback.groundedness,feedback.relevance,feedback.retrieval_relevance,execution_time,example_id,id
0,¿Puedo cambiar la hora de presentación de la p...,"No, no puedes cambiar la hora de presentación ...",[page_content='Presentación de la prueba de ad...,None,NO. La prueba de admisión se realiza de forma ...,True,True,True,True,2.249118,00d4b17d-38bd-4944-b58d-e23647884514,84fe034d-df51-46ff-a2e3-5cfcb0567664
1,En caso de ser admitido y no tener el acta de ...,Debes consultar con la División de Registro y ...,[page_content='entregar los documentos para la...,None,Se invita a consultar con la División de Regis...,True,True,True,True,2.271708,0d4fdcce-cd78-4f59-9d51-746bc92e032b,0d8d743d-6c05-47ac-8060-6a5a599070d7
2,En mi profesión no existe Registro Médico ni ...,Si en tu profesión no existe un Registro Médic...,[page_content='tipo de discapacidad en el form...,None,Los datos relacionados con la Tarjeta Profesio...,False,False,True,False,2.409597,1400d6a6-426e-49a8-acd4-f8bad69a0ea6,ead234b3-be1b-490d-b5d0-4a614d716255
3,¿Qué pasa si un aspirante no inscribe programa...,"Si el aspirante presentó la prueba, fue habili...",[page_content='34. Si el aspirante es convocad...,None,"Si el aspirante presentó la prueba, fue habili...",True,True,True,True,2.385233,209fbff6-a341-4e94-b638-530dc926a061,d4069c1e-fb1d-415d-943b-f7f218fe97e4
4,¿Qué es la admisión a pregrado?,La admisión a pregrado es el proceso mediante ...,[page_content='Admisión a pregrado\n\t\t\t\nLa...,None,La admisión es el proceso mediante el cual se ...,True,True,True,True,2.864530,29818cf6-0316-4b6b-875c-0bec7b2c51c9,afeac1cb-70eb-43d3-8d8d-b7868d229672
5,Si el programa al que deseo presentarme exige ...,Para inscribirte a la Prueba de Inglés Posgrad...,[page_content='Los aspirantes a los programas ...,None,Una vez usted realiza inscripción al proceso d...,False,True,True,True,2.308484,2b38f89d-b44b-42f3-894b-dcf8c80c1eef,a39c6537-4bf4-4b35-a0f5-00160dd31321
6,¿Cuál es la duración de la presentación de la ...,No sé.,[page_content='Presentación de la prueba de ad...,None,La prueba de admisión tiene una duración de 3 ...,False,False,False,False,1.988447,2f0beff2-61b6-4fd0-9c96-b652f63f7ce3,c49857e0-58ba-4679-9258-09f0ef82b2e5
7,En caso de ser admitido a un programa de posgr...,"Si es admitido a un programa de posgrado, el a...",[page_content='con la Coordinación Curricular ...,None,La información sobre trámites y calendario par...,True,True,True,True,2.798024,34960488-b533-4944-91cd-32bae8f8c205,cb6495ff-2a6c-4bf7-85f6-01cdca3daf77
8,No tengo número de registro del Examen de Esta...,Si eres extranjero y no tienes número de regis...,[page_content='debe identificarse con Cédula d...,None,"En el formulario electrónico de inscripción, e...",False,True,True,True,2.877291,3b174356-4c21-44f6-9a59-c546bb777949,5a937337-9361-4318-9e1f-03953a3d6a41
9,¿Debe el aspirante ser bachiller o haber prese...,"No, el aspirante no necesita ser bachiller ni ...","[page_content='undécimo, puedo inscribirme y p...",None,No. Para presentar la prueba de admisión el as...,True,True,True,True,2.114358,45125c16-301e-4b1d-a871-3b5a8dc43cc9,cee6a3de-60ff-4c43-8fb4-50d56f02d911


In [18]:
experiment_results_llama_emb.to_pandas()

,inputs.question,outputs.answer,outputs.documents,error,reference.answer,feedback.correctness,feedback.groundedness,feedback.relevance,feedback.retrieval_relevance,execution_time,example_id,id
0,¿Puedo cambiar la hora de presentación de la p...,No tengo información específica sobre cambios ...,[page_content='5. Consultar la citación y pres...,None,NO. La prueba de admisión se realiza de forma ...,False,False,True,False,1.876207,00d4b17d-38bd-4944-b58d-e23647884514,ed96e354-7209-4d35-b2ab-bf9599ca742d
1,En caso de ser admitido y no tener el acta de ...,"Lo siento, pero no tengo información específic...",[page_content='proceso y datos de contacto en ...,None,Se invita a consultar con la División de Regis...,False,False,False,False,2.402938,0d4fdcce-cd78-4f59-9d51-746bc92e032b,ea490f41-17c4-4326-a7de-54e5d664bf31
2,En mi profesión no existe Registro Médico ni ...,No tengo información específica sobre cómo lle...,[page_content='matrícula de los programas de p...,None,Los datos relacionados con la Tarjeta Profesio...,False,True,True,False,2.768351,1400d6a6-426e-49a8-acd4-f8bad69a0ea6,4eb90b26-35b0-4f79-98b7-09b3c7c496ca
3,¿Qué pasa si un aspirante no inscribe programa...,No tengo información específica sobre lo que s...,[page_content='participar en el proceso de ad...,None,"Si el aspirante presentó la prueba, fue habili...",False,False,False,False,1.942854,209fbff6-a341-4e94-b638-530dc926a061,0b6f97bc-bf53-4682-b930-30dfb8cc82bf
4,¿Qué es la admisión a pregrado?,"No tengo información específica sobre ""la admi...",[page_content='Es el proceso por el cual se se...,None,La admisión es el proceso mediante el cual se ...,False,False,False,False,2.779909,29818cf6-0316-4b6b-875c-0bec7b2c51c9,12420856-7b66-4916-8fa5-50d62e9931f9
5,Si el programa al que deseo presentarme exige ...,No tengo información específica sobre la Prueb...,[page_content='de la legitimidad de comprobant...,None,Una vez usted realiza inscripción al proceso d...,False,False,False,True,3.242282,2b38f89d-b44b-42f3-894b-dcf8c80c1eef,1778ac1d-1181-4547-9b0e-afd1cf8f4320
6,¿Cuál es la duración de la presentación de la ...,No tengo información sobre la duración de la p...,[page_content='11. Enviar los documentos reque...,None,La prueba de admisión tiene una duración de 3 ...,False,False,False,False,2.112587,2f0beff2-61b6-4fd0-9c96-b652f63f7ce3,ff5f9e1a-f2f0-4d96-9315-7faf0d2cb0b5
7,En caso de ser admitido a un programa de posgr...,No tengo información específica sobre el proce...,[page_content='Pasos que deben seguirse para r...,None,La información sobre trámites y calendario par...,False,False,False,False,1.432440,34960488-b533-4944-91cd-32bae8f8c205,bf82863d-1066-46ac-a1f0-b73f24ffb606
8,No tengo número de registro del Examen de Esta...,"Lo siento, no tengo información específica sob...",[page_content='5. Consultar la citación y pres...,None,"En el formulario electrónico de inscripción, e...",False,False,True,False,2.341460,3b174356-4c21-44f6-9a59-c546bb777949,722444f2-24be-4342-96c3-5c919da6fd5f
9,¿Debe el aspirante ser bachiller o haber prese...,No se menciona explícitamente en los documento...,[page_content='número asignado por el Institut...,None,No. Para presentar la prueba de admisión el as...,False,True,False,True,2.102417,45125c16-301e-4b1d-a871-3b5a8dc43cc9,982c298b-3361-491c-8ee2-3bd80fc19327
